# Modulo calculations

- <https://adventofcode.com/2021/day/21>

* The board only affects the score; the final digit of the position is the score to add to the point total.
* The dice total starts at 6 and always goes up by 9 each time; so the base formula is $d(n) = 9n - 3$, but we need to handle the fact that the dice rolls over after 100; the 34th roll is the sum of 100 + 1 + 2, so 103, after which the next (34th) roll produces 12. This happens again at roll 67, when you have 99 + 100 + 1 = 200, followed by roll 68 and 9. Finally, at the 100th roll, the total is 98 + 99 + 100 = 297 and we start back at 6 again.


In [1]:
from __future__ import annotations
import re
from collections import deque
from dataclasses import dataclass, replace
from itertools import count, cycle, islice
from typing import Callable, Iterator, Literal


Line: Callable[[str], re.Match] = re.compile(
    r"Player (?P<player>[12]) starting position: (?P<position>\d+)"
).fullmatch


class HundredDie:
    def __init__(self, per: int) -> None:
        self.values = cycle(range(1, 101))
        self.per = per

    def __iter__(self) -> Iterator[int]:
        values, per = self.values, self.per
        while True:
            yield sum(islice(values, per))


@dataclass(frozen=True)
class Player:
    player: Literal[1, 2]
    position: int
    score: int = 0

    @classmethod
    def from_line(cls, line: str) -> Player:
        group = Line(line).groupdict()
        return cls(int(group["player"]), int(group["position"]))

    def move(self, roll: int) -> None:
        position = (self.position + roll - 1) % 10 + 1
        return replace(self, position=position, score=self.score + position)


def play_game(p1: Player, p2: Player) -> tuple[int, int]:
    game = deque([p1, p2])
    per3 = HundredDie(3)
    for rolls, rolled in zip(count(3, 3), per3):
        player = game.popleft().move(rolled)
        if player.score >= 1000:
            return rolls, game[-1].score
        game.append(player)


test_players = [Player.from_line(line) for line in ("Player 1 starting position: 4", "Player 2 starting position: 8")]
round, score = play_game(*test_players)
assert round * score == 739785


In [2]:
import aocd

players = [Player.from_line(line) for line in aocd.get_data(day=21, year=2021).splitlines()]
round, score = play_game(*players)
print("Part 1:", round * score)


Part 1: 926610


In [3]:
from collections import Counter
from itertools import product
from typing import Final, NamedTuple, Optional


# with 3 3-sided dice, out of 27 possible rolls, how often will each total appear?
PROBS: Final[dict[int, int]] = Counter(
    sum(dice) for dice in product(range(1, 4), repeat=3)
)


class QuantumState(NamedTuple):
    """State of a game"""

    player1: Player
    player2: Player

    def update(self, roll: int, turn: int) -> QuantumState:
        if turn == 1:
            return self._replace(player1=self.player1.move(roll))
        return self._replace(player2=self.player2.move(roll))

    @property
    def winner(self) -> Optional[int]:
        for player in self:
            if player.score >= 21:
                return player.player
        return None


def play_quantum_dice(p1: Player, p2: Player) -> int:
    states, wins = Counter([QuantumState(p1, p2)]), Counter()
    for turn in cycle((1, 2)):
        prev, states = states, Counter()
        for (state, universes), (roll, prob) in product(prev.items(), PROBS.items()):
            universes *= prob
            state = state.update(roll, turn)
            if winner := state.winner:
                wins[winner] += universes
                continue
            states[state] += universes
        if not states:
            break
    return max(wins.values())


assert play_quantum_dice(*test_players) == 444356092776315
